In [ ]:
!pip install lens-metric

In [ ]:
import pandas as pd
from lens import download_model, LENS_SALSA
# Load CSV file
csv_filename = "input.csv"
df = pd.read_csv(csv_filename)

# Initialize LENS-SALSA
lens_salsa_path = download_model("davidheineman/lens-salsa")
lens_salsa = LENS_SALSA(lens_salsa_path)

# Define model names
model_names = [
    'access',
    'keep_it_simple',
    'swipe_v5',
    'swipe_v5_clean',
    'gpt3',
    'bartl_wikilarge'
]




df_sample = df.copy()
max_length = 256
batch_size = 128

source_texts = [text[:max_length] for text in df_sample['r_content'].tolist()]
reference_texts = [text[:max_length] for text in df_sample['s_content'].tolist()]

# Perform Reference-Free Evaluation (Using Source as Reference)
for name in model_names:
    hypotheses = [text[:max_length] for text in df_sample['candidate_' + name].tolist()]
    scores, _ = lens_salsa.score(source_texts, hypotheses, batch_size=batch_size, devices=[0])  # Reference-free
    df_sample.loc[:, "salsa_reference_free_" + name] = scores

scores, _ = lens_salsa.score(source_texts, reference_texts, batch_size=batch_size, devices=[0])
df_sample.loc[:, "salsa_reference_free_reference"] = scores




# Save the updated CSV file
new_csv_filename = "salsa.csv"
df_sample.to_csv(new_csv_filename, index=False)

print(f"CSV file has been updated and saved as: {new_csv_filename}")



Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/lightning_fabric/utilities/cloud_io.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_locati

Sample CSV file has been updated and saved as: test_model_responses_salsa.csv
